<a href="https://colab.research.google.com/github/jcandane/CI_Theory/blob/main/FCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Goal of this notebook is to introduce a standalone FCI program in Python.

## Import Libraries, set defaults

In [1]:
!pip install pyscf

import numpy as np
from itertools import combinations, permutations
from pyscf import fci, ao2mo, scf, gto

π = np.pi
α = 0.007297352
c = 1.0/α
np.set_printoptions(precision=4, linewidth=200, threshold=2000, suppress=True)

     |████████████████████████████████| 37.5 MB 1.7 MB/s 


## Let's define a few useful python functions:

In [2]:
def givenΛgetB(ΛA, ΛB, N_mo):
  "Given Λ (i occupied orbitals for each determinant) get B (binary rep.)"

  Binary  = np.zeros((ΛA.shape[0], 2, N_mo), dtype=np.int8)
  for I in range(len(Binary)):
      Binary[I, 0, ΛA[I,:]] = 1
      Binary[I, 1, ΛB[I,:]] = 1

  return Binary

def SpinOuterProduct(A, B, stack=False):
  ΛA = np.einsum("Ii, J -> IJi", A, np.ones(B.shape[0], dtype=np.int8)).reshape( (A.shape[0]*B.shape[0], A.shape[1]) )
  ΛB = np.einsum("Ii, J -> JIi", B, np.ones(A.shape[0], dtype=np.int8)).reshape( (A.shape[0]*B.shape[0], B.shape[1]) )
  
  if stack:
    return np.array([ΛA,ΛB])
  else:
    return ΛA, ΛB

def get_SO_matrix(uhf_pyscf, SF=False, H1=None, H2=None):
    """ Given a PySCF uhf object get SO Matrices """

    Ca, Cb = (uhf_pyscf).mo_coeff
    S = (uhf_pyscf.mol).intor("int1e_ovlp")
    eig, v = np.linalg.eigh(S)
    A = (v) @ np.diag(eig**(-0.5)) @ np.linalg.inv(v) 
    H = uhf_pyscf.get_hcore() 

    n = Ca.shape[1]
    eri_aa = (ao2mo.general( (uhf_pyscf)._eri , (Ca, Ca, Ca, Ca), compact=False)).reshape((n,n,n,n), order="C")
    eri_aa -= eri_aa.swapaxes(1,3)
    eri_bb = (ao2mo.general( (uhf_pyscf)._eri , (Cb, Cb, Cb, Cb), compact=False)).reshape((n,n,n,n), order="C")
    eri_bb -= eri_bb.swapaxes(1,3)
    eri_ab = (ao2mo.general( (uhf_pyscf)._eri , (Ca, Ca, Cb, Cb), compact=False)).reshape((n,n,n,n), order="C")
    #eri_ba = (1.*eri_ab).swapaxes(0,3).swapaxes(1,2) ## !! caution depends on symmetry
    eri_ba = (ao2mo.general( (uhf_pyscf)._eri , (Cb, Cb, Ca, Ca), compact=False)).reshape((n,n,n,n), order="C")
    H2 = np.stack(( np.stack((eri_aa, eri_ab)), np.stack((eri_ba, eri_bb)) ))

    H1 = np.asarray([np.einsum("AB, Ap, Bq -> pq", H, Ca, Ca), np.einsum("AB, Ap, Bq -> pq", H, Cb, Cb)])

    if SF:
      eri_abab = (ao2mo.general( (uhf_pyscf)._eri , (Ca, Cb, Ca, Cb), compact=False)).reshape((n,n,n,n), order="C")
      eri_abba = (ao2mo.general( (uhf_pyscf)._eri , (Ca, Cb, Cb, Ca), compact=False)).reshape((n,n,n,n), order="C")
      eri_baab = (ao2mo.general( (uhf_pyscf)._eri , (Cb, Ca, Ca, Cb), compact=False)).reshape((n,n,n,n), order="C")
      eri_baba = (ao2mo.general( (uhf_pyscf)._eri , (Cb, Ca, Cb, Ca), compact=False)).reshape((n,n,n,n), order="C")
      H2_SF = np.stack(( np.stack((eri_abab, eri_abba)), np.stack((eri_baab, eri_baba)) ))
      return H1, H2, H2_SF

    else:
      return H1, H2

def get_excitation_op(i, j, binary, sign, spin=0):
    Difference = binary[i,spin] - binary[j, spin]
    #a_t  =    (Difference[i, j, spin] + 0.5).astype(np.int8)
    #a    = -1*(Difference[i, j, spin] - 0.5).astype(np.int8)
    a_t =    (Difference + 0.5).astype(np.int8)
    a   = -1*(Difference - 0.5).astype(np.int8)
    if np.sum(a[0]) > 1: ### this is a double excitation
        å_t = 1*a_t ## make copy
        å_t[ np.arange(len(å_t)),(å_t!=0).argmax(axis=1) ] = 0 ## zero first 1
        a_t = np.abs(å_t - a_t) ## absolute difference from orginal
        a_t = np.asarray([sign[j, spin]*å_t,sign[j, spin]*a_t]) ## stack

        å = 1*a ## make copy
        å[ np.arange(len(å)),(å!=0).argmax(axis=1) ] = 0 ## zero first 1
        a = np.abs(å - a) ## absolute difference from orginal
        a = np.asarray([sign[i, spin]*å,sign[i, spin]*a]) ## stack

        return a_t, a

    return sign[j, spin]*a_t, sign[i, spin]*a

## Let's now do a PySCF HF Calculation

In [3]:
mol = gto.M(atom='He 0.0 0.0 0.0; H 1.5 0.0 0.0; He 0.0 1.5 0.0; N 0.0, 0.0, 1.5', spin=0, basis="sto-3g")
uhf = scf.UHF(mol)
uhf.kernel()

### information about occupation
O_sp   = np.asarray((uhf).mo_occ, dtype=np.int8)
N_s    = np.einsum("sp -> s", O_sp)

converged SCF energy = -59.3998556537129  <S^2> = 1.5861251e-06  2S+1 = 1.0000032


## Configuration Generation (fCI)

In [4]:
N   = O_sp.shape[1]
Λ_α = np.asarray( list(combinations(  np.arange(0, N, 1, dtype=np.int8)  , N_s[0] ) ) ) 
Λ_β = np.asarray( list(combinations(  np.arange(0, N, 1, dtype=np.int8)  , N_s[1] ) ) ) 
ΛA, ΛB = SpinOuterProduct(Λ_α, Λ_β)
Binary = givenΛgetB(ΛA, ΛB, N)

Determine Determinant Signs

In [5]:
sign  = np.cumsum( Binary, axis=2)
for I in range(len(Binary)):
    iia = np.where( Binary[I,0] == 1)[0]
    iib = np.where( Binary[I,1] == 1)[0]
    sign[I, 0, iia] = np.arange(0, len(iia), 1)
    sign[I, 1, iib] = np.arange(0, len(iib), 1)

sign = ( (-1)**(sign) ).astype(np.int8)

## Let's now get SO Matrix Elements

In [6]:
H1, H2 = get_SO_matrix(uhf)

## Let's now get CI Matrix Elements

Slater-Condon Rule 0 & 3

In [7]:
## Rule 0 & 3
H_CI  = np.einsum("Spp, ISp -> I", H1, Binary, optimize=True)
H_CI += np.einsum("STppqq, ISp, ITq -> I", H2, Binary, Binary, optimize=True)/2
H_CI  = np.diag(H_CI)

Calclate Spin Differences between all determinants to determine the applicable rule.

In [8]:
SpinDifference = np.sum( np.abs(Binary[:, None, :, :] - Binary[None, :, :, :]), axis=3)//2

Slater-Condon Rule 1

In [9]:
## indices for 1-difference
I_A, J_A = np.where( np.all(SpinDifference==np.array([1,0], dtype=np.int8), axis=2) )
I_B, J_B = np.where( np.all(SpinDifference==np.array([0,1], dtype=np.int8), axis=2) )

### get excitation operators
a_t , a  = get_excitation_op(I_A , J_A , Binary, sign, spin=0)
b_t , b  = get_excitation_op(I_B , J_B , Binary, sign, spin=1)
ca       = ((Binary[I_A,0,:] + Binary[J_A,0,:])/2).astype(np.int8)
cb       = ((Binary[I_B,1,:] + Binary[J_B,1,:])/2).astype(np.int8)

## Rule 1
H_CI[I_A , J_A ] -= np.einsum("pq, Kp, Kq -> K", H1[0], a_t, a, optimize=True)
H_CI[I_A , J_A ] -= np.einsum("pqrr, Kp, Kq, Kr -> K", H2[0,0], a_t, a, ca, optimize=True)
H_CI[I_A , J_A ] -= np.einsum("pqrr, Kp, Kq, Kr -> K", H2[0,1], a_t, a, Binary[I_A,1], optimize=True)

H_CI[I_B , J_B ] -= np.einsum("pq, Kp, Kq -> K", H1[1], b_t, b, optimize=True)
H_CI[I_B , J_B ] -= np.einsum("pqrr, Kp, Kq, Kr -> K", H2[1,1], b_t, b, cb, optimize=True)
H_CI[I_B , J_B ] -= np.einsum("pqrr, Kp, Kq, Kr -> K", H2[1,0], b_t, b, Binary[I_B,0], optimize=True)

Slater-Condon Rule 2

In [10]:
## indices for 2-differences
I_AA, J_AA = np.where( np.all(SpinDifference==np.array([2,0], dtype=np.int8), axis=2) )
I_BB, J_BB = np.where( np.all(SpinDifference==np.array([0,2], dtype=np.int8), axis=2) )
I_AB, J_AB = np.where( np.all(SpinDifference==np.array([1,1], dtype=np.int8), axis=2) )

aa_t, aa = get_excitation_op(I_AA, J_AA, Binary, sign, spin=0)
bb_t, bb = get_excitation_op(I_BB, J_BB, Binary, sign, spin=1)
ab_t, ab = get_excitation_op(I_AB, J_AB, Binary, sign, spin=0)
ba_t, ba = get_excitation_op(I_AB, J_AB, Binary, sign, spin=1)

## Rule 2
H_CI[I_AA, J_AA]  = np.einsum("pqrs, Kp, Kq, Kr, Ks -> K", H2[0,0], aa_t[0], aa[0], aa_t[1], aa[1], optimize=True)
H_CI[I_BB, J_BB]  = np.einsum("pqrs, Kp, Kq, Kr, Ks -> K", H2[1,1], bb_t[0], bb[0], bb_t[1], bb[1], optimize=True)
H_CI[I_AB, J_AB]  = np.einsum("pqrs, Kp, Kq, Kr, Ks -> K", H2[0,1], ab_t, ab, ba_t, ba, optimize=True)

## Calculation

### PySCF FCI

In [11]:
cisolver = fci.FCI(uhf,singlet=False)
cisolver.nroots = 100 # 100
cisolver.spin = 0
cisolver.davidson_only = False
pyscf_fci_energy = cisolver.kernel()[0]

### Comparison

In [15]:
nuclear_rep = uhf.energy_nuc()
e_fci_my, X_IJ = np.linalg.eigh(H_CI)

print("PySCF : " + str(pyscf_fci_energy[0] ))
print("Mine  : " + str( e_fci_my[0] + nuclear_rep ))

from scipy.sparse.linalg import eigs
vals, vecs = eigs(H_CI, k=6) ## for sparse matrix get 1st 6 eigenvectors and eigenvalues
print("MineSp: " + str( vals[0].real + nuclear_rep )) 

PySCF : -59.687488249646734
Mine  : -59.68748824964679
MineSp: -59.68748824964665
